<a href="https://colab.research.google.com/github/cryptopatience/Main-code/blob/main/Mag_7_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 설치 및 임포트
!pip install yfinance pandas numpy plotly -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
warnings.filterwarnings('ignore')

# 구글 코랩용 렌더러 설정
pio.renderers.default = 'colab'

# ==================== Magnificent Seven 정의 ====================

MAG7_STOCKS = {
    'AAPL': {
        'name': 'Apple Inc.',
        'description': '아이폰, 생태계, 온디바이스 AI',
        'sector': 'Technology',
        'industry': 'Consumer Electronics'
    },
    'MSFT': {
        'name': 'Microsoft Corporation',
        'description': '클라우드(Azure), 생성형 AI (OpenAI 대주주)',
        'sector': 'Technology',
        'industry': 'Software'
    },
    'GOOGL': {
        'name': 'Alphabet Inc.',
        'description': '구글 검색, 유튜브, AI (Gemini)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'AMZN': {
        'name': 'Amazon.com Inc.',
        'description': '전자상거래, 클라우드(AWS) 1위',
        'sector': 'Consumer Cyclical',
        'industry': 'Internet Retail'
    },
    'NVDA': {
        'name': 'NVIDIA Corporation',
        'description': 'AI 반도체(GPU) 독점적 지배자',
        'sector': 'Technology',
        'industry': 'Semiconductors'
    },
    'META': {
        'name': 'Meta Platforms Inc.',
        'description': '페이스북, 인스타그램, AI(Llama)',
        'sector': 'Communication Services',
        'industry': 'Internet Content & Information'
    },
    'TSLA': {
        'name': 'Tesla Inc.',
        'description': '전기차, 자율주행, 로봇',
        'sector': 'Consumer Cyclical',
        'industry': 'Auto Manufacturers'
    }
}

print("=" * 80)
print("🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석")
print("=" * 80)
print("\n빅테크 7대 기업 리스트:")
for ticker, info in MAG7_STOCKS.items():
    print(f"  • {ticker:6} - {info['name']:30} | {info['description']}")
print("=" * 80)

def get_current_quarter_start():
    """현재 분기 시작일 계산"""
    now = datetime.now()
    quarter = (now.month - 1) // 3
    quarter_start_month = quarter * 3 + 1
    quarter_start = datetime(now.year, quarter_start_month, 1)
    return quarter_start

def calculate_anchored_vwap(df):
    """Anchored VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['TP_Volume'] = df['Typical_Price'] * df['Volume']
    df['Cumulative_TP_Volume'] = df['TP_Volume'].cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Anchored_VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df

def get_quarterly_vwap_analysis(ticker):
    """분기별 Anchored VWAP 분석"""
    try:
        quarter_start = get_current_quarter_start()
        end_date = datetime.now()
        quarter_num = (quarter_start.month - 1) // 3 + 1

        stock = yf.Ticker(ticker)
        df = stock.history(start=quarter_start, end=end_date)

        if df.empty or len(df) < 5:
            return None

        df = calculate_anchored_vwap(df)

        current_price = df['Close'].iloc[-1]
        current_vwap = df['Anchored_VWAP'].iloc[-1]
        above_vwap_ratio = (df['Close'] > df['Anchored_VWAP']).sum() / len(df) * 100
        recent_5days_avg = df['Close'].tail(5).mean()
        recent_10days_avg = df['Close'].tail(10).mean()

        recent_20 = df['Close'].tail(min(20, len(df)))
        uptrend_strength = (recent_20.diff() > 0).sum() / len(recent_20) * 100 if len(recent_20) > 1 else 50

        recent_volume = df['Volume'].tail(5).mean()
        avg_volume = df['Volume'].mean()
        volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 1

        info = stock.info
        quarter_start_price = df['Close'].iloc[0]
        quarter_return = ((current_price - quarter_start_price) / quarter_start_price * 100)

        return {
            'Ticker': ticker,
            'Company': MAG7_STOCKS[ticker]['name'],
            'Description': MAG7_STOCKS[ticker]['description'],
            'Sector': MAG7_STOCKS[ticker]['sector'],
            'Quarter': f'{quarter_start.year} Q{quarter_num}',
            'Quarter_Start_Date': quarter_start.strftime('%Y-%m-%d'),
            'Trading_Days': len(df),
            'Current_Price': round(current_price, 2),
            'Anchored_VWAP': round(current_vwap, 2),
            'Quarter_Start_Price': round(quarter_start_price, 2),
            'Quarter_Return_%': round(quarter_return, 2),
            'Price_vs_VWAP_%': round((current_price - current_vwap) / current_vwap * 100, 2),
            'Above_VWAP_Days_%': round(above_vwap_ratio, 1),
            'Recent_5D_Avg': round(recent_5days_avg, 2),
            'Recent_10D_Avg': round(recent_10days_avg, 2),
            'Uptrend_Strength_%': round(uptrend_strength, 1),
            'Volume_Ratio': round(volume_ratio, 2),
            'Is_Above_VWAP': current_price > current_vwap,
            'Market_Cap': info.get('marketCap', 0),
            'Buy_Signal_Score': 0
        }

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None

# 분기 정보
quarter_start = get_current_quarter_start()
quarter_num = (quarter_start.month - 1) // 3 + 1

print("\n" + "=" * 80)
print(f"🎯 Anchored VWAP 분석: {quarter_start.year} Q{quarter_num}")
print(f"📍 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"📅 현재까지 거래일: {(datetime.now() - quarter_start).days}일")
print("=" * 80)

# MAG 7 분석
results = []
mag7_tickers = list(MAG7_STOCKS.keys())

print("\n🔍 MAG 7 종목 분석 중...\n")

for ticker in mag7_tickers:
    print(f"분석 중: {ticker} ({MAG7_STOCKS[ticker]['name']})...", end=" ")
    result = get_quarterly_vwap_analysis(ticker)
    if result:
        results.append(result)
        print("✓")
    else:
        print("✗")

df_results = pd.DataFrame(results)

# 매수 신호 점수 계산
def calculate_buy_score(row):
    score = 0
    if row['Is_Above_VWAP']:
        score += 30

    price_diff = row['Price_vs_VWAP_%']
    if 0 < price_diff <= 5:
        score += 20
    elif 5 < price_diff <= 10:
        score += 10
    elif price_diff > 10:
        score += 5

    if row['Above_VWAP_Days_%'] >= 80:
        score += 20
    elif row['Above_VWAP_Days_%'] >= 60:
        score += 15
    elif row['Above_VWAP_Days_%'] >= 40:
        score += 10

    if row['Uptrend_Strength_%'] >= 60:
        score += 15
    elif row['Uptrend_Strength_%'] >= 50:
        score += 10

    if row['Volume_Ratio'] >= 1.2:
        score += 15
    elif row['Volume_Ratio'] >= 1.0:
        score += 10

    return min(score, 100)

df_results['Buy_Signal_Score'] = df_results.apply(calculate_buy_score, axis=1)
df_results = df_results.sort_values('Buy_Signal_Score', ascending=False)

# 시가총액 추가
df_results['Market_Cap_Trillion'] = (df_results['Market_Cap'] / 1e12).round(3)

print("\n" + "=" * 80)
print("📊 MAGNIFICENT SEVEN - 실시간 분석 결과")
print("=" * 80)

display_cols = ['Ticker', 'Company', 'Description', 'Current_Price', 'Anchored_VWAP',
                'Price_vs_VWAP_%', 'Quarter_Return_%', 'Market_Cap_Trillion', 'Buy_Signal_Score']
print(df_results[display_cols].to_string(index=False))

print("\n" + "=" * 80)
print("🏆 MAG 7 투자 추천 순위")
print("=" * 80)

for idx, row in df_results.iterrows():
    rank = df_results.index.get_loc(idx) + 1

    if rank == 1:
        medal = "🥇"
    elif rank == 2:
        medal = "🥈"
    elif rank == 3:
        medal = "🥉"
    else:
        medal = f"  {rank}"

    print(f"\n{medal} {row['Ticker']} - {row['Company']}")
    print(f"    🎯 {row['Description']}")
    print(f"    💰 시가총액: ${row['Market_Cap_Trillion']:.2f}T")
    print(f"    📈 현재가: ${row['Current_Price']} | VWAP: ${row['Anchored_VWAP']}")
    print(f"    📊 VWAP 대비: {row['Price_vs_VWAP_%']:+.2f}% | 분기 수익률: {row['Quarter_Return_%']:+.2f}%")
    print(f"    🎲 VWAP 위 거래일: {row['Above_VWAP_Days_%']}% | 추세 강도: {row['Uptrend_Strength_%']}%")
    print(f"    ⭐ 매수 신호 점수: {row['Buy_Signal_Score']}/100", end="")

    if row['Buy_Signal_Score'] >= 80:
        print(" → 💚 강력 매수 추천")
    elif row['Buy_Signal_Score'] >= 60:
        print(" → 💛 눌림목 대기")
    else:
        print(" → 💙 관찰 필요")

# ==================== 펀더멘털 분석 ====================

def get_comprehensive_analysis(ticker):
    """종목별 펀더멘털 분석"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))

        def safe_get(key, default='N/A', multiplier=1, format_str=None):
            value = info.get(key)
            if value is None or value == 'N/A':
                return default
            try:
                if format_str:
                    return format_str.format(value * multiplier)
                return value * multiplier
            except:
                return default

        recommendation_map = {
            'buy': '매수', 'strong buy': '적극 매수', 'hold': '보유',
            'sell': '매도', 'strong sell': '적극 매도'
        }
        rec_key = info.get('recommendationKey', 'N/A').lower()
        recommendation_kr = recommendation_map.get(rec_key, rec_key.upper())

        target_price = safe_get('targetMeanPrice', 0)
        upside = 'N/A'
        if target_price and target_price > 0 and current_price > 0:
            upside = f"{((target_price / current_price) - 1) * 100:.2f}%"

        report = {
            "===== [기본 정보] =====": "",
            "종목명": info.get('longName', ticker),
            "비즈니스": MAG7_STOCKS[ticker]['description'],
            "섹터": info.get('sector', 'N/A'),
            "현재가": f"${current_price:.2f}" if current_price else "N/A",
            "시가총액": f"${safe_get('marketCap', 0) / 1e12:.3f}T" if safe_get('marketCap', 0) else "N/A",

            "===== [가치 평가] =====": "",
            "PER (주가수익비율)": f"{safe_get('trailingPE', 0):.2f}" if safe_get('trailingPE') != 'N/A' else "N/A",
            "Forward PER": f"{safe_get('forwardPE', 0):.2f}" if safe_get('forwardPE') != 'N/A' else "N/A",
            "PBR": f"{safe_get('priceToBook', 0):.2f}" if safe_get('priceToBook') != 'N/A' else "N/A",
            "PEG Ratio": f"{safe_get('pegRatio', 0):.2f}" if safe_get('pegRatio') != 'N/A' else "N/A",

            "===== [수익성 & 재무] =====": "",
            "ROE": safe_get('returnOnEquity', default='N/A', multiplier=100, format_str="{:.2f}%"),
            "영업이익률": safe_get('operatingMargins', default='N/A', multiplier=100, format_str="{:.2f}%"),
            "순이익률": safe_get('profitMargins', default='N/A', multiplier=100, format_str="{:.2f}%"),
            "부채비율": f"{safe_get('debtToEquity', 0):.2f}%" if safe_get('debtToEquity') != 'N/A' else "N/A",

            "===== [성장성] =====": "",
            "매출 성장률": safe_get('revenueGrowth', default='N/A', multiplier=100, format_str="{:.2f}%"),
            "이익 성장률": safe_get('earningsGrowth', default='N/A', multiplier=100, format_str="{:.2f}%"),

            "===== [월가 컨센서스] =====": "",
            "투자의견": recommendation_kr,
            "목표주가": f"${target_price:.2f}" if target_price else "N/A",
            "상승여력": upside,
        }

        return report

    except Exception as e:
        return {"Error": f"분석 실패: {str(e)}"}

print("\n\n" + "=" * 80)
print("📋 MAG 7 - 펀더멘털 상세 분석")
print("=" * 80)

for idx, row in df_results.iterrows():
    ticker = row['Ticker']
    rank = df_results.index.get_loc(idx) + 1

    print(f"\n{'='*80}")
    print(f"[순위 {rank}] 📊 {ticker} - {row['Company']}")
    print("="*80)

    analysis = get_comprehensive_analysis(ticker)

    for key, value in analysis.items():
        if "=====" in key:
            print(f"\n{value}{key}")
        else:
            print(f"  {key:<20}: {value}")

    print("-" * 80)

# ==================== 차트 생성 ====================

print("\n📈 인터랙티브 차트 생성 중...")

# 차트 1: 매수 신호 점수
fig1 = go.Figure()

colors_score = ['#00C853' if x >= 80 else '#FFD600' if x >= 60 else '#2196F3'
                for x in df_results['Buy_Signal_Score']]

fig1.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['Buy_Signal_Score'],
    orientation='h',
    marker=dict(color=colors_score),
    text=df_results['Buy_Signal_Score'],
    textposition='auto',
    hovertemplate='<b>%{y}</b><br>매수 점수: %{x}/100<extra></extra>'
))

fig1.update_layout(
    title=f'🎯 MAG 7 매수 신호 점수 (Anchored VWAP - {quarter_start.year} Q{quarter_num})',
    xaxis_title='매수 신호 점수',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

fig1.show()

# 차트 2: VWAP 대비 가격 위치
fig2 = go.Figure()

colors = ['green' if x > 0 else 'red' for x in df_results['Price_vs_VWAP_%']]

fig2.add_trace(go.Bar(
    y=df_results['Ticker'],
    x=df_results['Price_vs_VWAP_%'],
    orientation='h',
    marker=dict(color=colors),
    text=df_results['Price_vs_VWAP_%'].round(1),
    textposition='auto'
))

fig2.add_vline(x=0, line_dash="dash", line_color="black", line_width=2)

fig2.update_layout(
    title=f'📊 MAG 7 - Anchored VWAP 대비 가격 위치 ({quarter_start.year} Q{quarter_num})',
    xaxis_title='VWAP 대비 차이 (%)',
    yaxis_title='종목',
    height=500,
    template='plotly_white'
)

fig2.show()

# 차트 3: 시가총액 vs 분기 수익률
fig3 = px.scatter(
    df_results,
    x='Market_Cap_Trillion',
    y='Quarter_Return_%',
    size='Buy_Signal_Score',
    color='Buy_Signal_Score',
    hover_data=['Ticker', 'Company', 'Description'],
    text='Ticker',
    color_continuous_scale='RdYlGn',
    title=f'💰 MAG 7 - 시가총액 vs 분기 수익률 ({quarter_start.year} Q{quarter_num})',
    labels={'Market_Cap_Trillion': '시가총액 (Trillion USD)',
            'Quarter_Return_%': '분기 수익률 (%)'}
)

fig3.update_traces(textposition='top center', textfont_size=12)
fig3.update_layout(height=600)
fig3.show()

# 차트 4: MAG 7 1년 차트
print("\n📈 MAG 7 종목별 1년 차트 생성 중...")

def get_quarterly_anchors(start_date, end_date):
    """분기 시작일 계산"""
    quarters = []
    current = start_date

    while current <= end_date:
        year = current.year
        month = current.month
        quarter_start_month = ((month - 1) // 3) * 3 + 1
        quarter_start = datetime(year, quarter_start_month, 1)

        if quarter_start not in [q[0] for q in quarters] and quarter_start >= start_date:
            quarter_num = (quarter_start_month - 1) // 3 + 1
            quarters.append((quarter_start, f"Q{quarter_num} {year}"))

        if month >= 10:
            current = datetime(year + 1, 1, 1)
        else:
            current = datetime(year, quarter_start_month + 3, 1)

    return quarters

def calculate_multiple_anchored_vwaps(df, anchor_points):
    """여러 분기 VWAP 계산"""
    df = df.copy()
    df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3
    vwap_columns = {}
    df_tz = df.index.tz if hasattr(df.index, 'tz') else None

    for anchor_date, quarter_label in anchor_points:
        try:
            if df_tz is not None:
                anchor_date_tz = pd.Timestamp(anchor_date).tz_localize(df_tz)
            else:
                anchor_date_tz = pd.Timestamp(anchor_date)

            mask = df.index >= anchor_date_tz
            if mask.sum() == 0:
                continue

            df_period = df[mask].copy()
            tp_volume = (df_period['Typical_Price'] * df_period['Volume']).cumsum()
            cumulative_volume = df_period['Volume'].cumsum()
            vwap = tp_volume / cumulative_volume

            vwap_full = pd.Series(index=df.index, dtype=float)
            vwap_full[mask] = vwap.values
            vwap_columns[quarter_label] = vwap_full

        except Exception as e:
            continue

    return vwap_columns

# 각 종목별 차트
for idx, row in df_results.iterrows():
    ticker = row['Ticker']
    rank = df_results.index.get_loc(idx) + 1

    print(f"\n[{rank}/7] 차트 생성: {ticker}...", end=" ")

    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)

        stock = yf.Ticker(ticker)
        df_1year = stock.history(start=start_date, end=end_date)

        if df_1year.empty:
            print("✗ 데이터 없음")
            continue

        quarter_anchors = get_quarterly_anchors(start_date, end_date)
        vwap_dict = calculate_multiple_anchored_vwaps(df_1year, quarter_anchors)

        fig = go.Figure()

        # 캔들스틱
        fig.add_trace(go.Candlestick(
            x=df_1year.index,
            open=df_1year['Open'],
            high=df_1year['High'],
            low=df_1year['Low'],
            close=df_1year['Close'],
            name='Price',
            increasing_line_color='#26a69a',
            decreasing_line_color='#ef5350'
        ))

        # 분기별 VWAP
        colors_vwap = ['#2196F3', '#FF9800', '#4CAF50', '#9C27B0', '#F44336']

        for idx_vwap, (quarter_label, vwap_series) in enumerate(vwap_dict.items()):
            color = colors_vwap[idx_vwap % len(colors_vwap)]
            vwap_clean = vwap_series.dropna()

            if len(vwap_clean) > 0:
                fig.add_trace(go.Scatter(
                    x=vwap_clean.index,
                    y=vwap_clean,
                    mode='lines',
                    name=f'VWAP {quarter_label}',
                    line=dict(color=color, width=2),
                    opacity=0.7
                ))

        # 현재 분기 강조
        if vwap_dict:
            latest_quarter = list(vwap_dict.keys())[-1]
            latest_vwap = vwap_dict[latest_quarter].dropna()

            if len(latest_vwap) > 0:
                fig.add_trace(go.Scatter(
                    x=latest_vwap.index,
                    y=latest_vwap,
                    mode='lines',
                    name=f'✨ Current {latest_quarter}',
                    line=dict(color='red', width=3),
                    opacity=1
                ))

        # 거래량
        fig.add_trace(go.Bar(
            x=df_1year.index,
            y=df_1year['Volume'],
            name='Volume',
            marker_color='rgba(128, 128, 128, 0.3)',
            yaxis='y2'
        ))

        fig.update_layout(
            title={
                'text': f"<b>#{rank} {ticker} - {row['Company']}</b><br>" +
                        f"<sub>{row['Description']}</sub><br>" +
                        f"<sub>매수 점수: {row['Buy_Signal_Score']}/100 | " +
                        f"현재가: ${row['Current_Price']} | " +
                        f"VWAP: ${row['Anchored_VWAP']} | " +
                        f"차이: {row['Price_vs_VWAP_%']:+.2f}%</sub>",
                'x': 0.5,
                'xanchor': 'center'
            },
            xaxis=dict(title='날짜', rangeslider=dict(visible=False)),
            yaxis=dict(title='가격 (USD)', side='right'),
            yaxis2=dict(
                title='거래량',
                overlaying='y',
                side='left',
                showgrid=False,
                range=[0, df_1year['Volume'].max() * 4]
            ),
            hovermode='x unified',
            height=700,
            template='plotly_white'
        )

        # 분기 세로선
        df_tz = df_1year.index.tz if hasattr(df_1year.index, 'tz') else None

        for anchor_date, quarter_label in quarter_anchors:
            if df_tz is not None:
                anchor_display = pd.Timestamp(anchor_date).tz_localize(df_tz)
            else:
                anchor_display = anchor_date

            fig.add_shape(
                type="line",
                x0=anchor_display,
                x1=anchor_display,
                y0=0,
                y1=1,
                yref="paper",
                line=dict(color="gray", width=1, dash="dash"),
                opacity=0.5
            )

            fig.add_annotation(
                x=anchor_display,
                y=1,
                yref="paper",
                text=quarter_label,
                showarrow=False,
                yshift=10,
                font=dict(size=10, color="gray")
            )

        fig.show()
        print("✓")

    except Exception as e:
        print(f"✗ 실패: {str(e)}")

# 차트 5: MAG 7 비교 차트
print("\n📊 MAG 7 수익률 비교 차트 생성 중...")

fig_compare = go.Figure()

for ticker in mag7_tickers:
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=365)

        stock = yf.Ticker(ticker)
        df_1year = stock.history(start=start_date, end=end_date)

        if df_1year.empty:
            continue

        normalized_price = (df_1year['Close'] / df_1year['Close'].iloc[0]) * 100

        fig_compare.add_trace(go.Scatter(
            x=df_1year.index,
            y=normalized_price,
            mode='lines',
            name=ticker,
            line=dict(width=2.5),
            hovertemplate=f'<b>{ticker}</b><br>날짜: %{{x}}<br>수익률: %{{y:.2f}}%<extra></extra>'
        ))

    except:
        continue

fig_compare.update_layout(
    title='🏆 MAGNIFICENT SEVEN - 1년 수익률 비교 (정규화)',
    xaxis_title='날짜',
    yaxis_title='정규화 가격 (시작점 = 100)',
    hovermode='x unified',
    height=600,
    showlegend=True,
    template='plotly_white'
)

fig_compare.add_hline(y=100, line_dash="dash", line_color="gray",
                       annotation_text="시작점", annotation_position="right")

fig_compare.show()

print("✓ 비교 차트 완성")

# ==================== 5개년 성장률 분석 ====================

print("\n" + "=" * 80)
print("📈 MAG 7 - 최근 5개년 성장률 분석")
print("=" * 80)

def get_5year_growth_metrics(ticker):
    """5개년 성장률 데이터 수집"""
    try:
        stock = yf.Ticker(ticker)
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        cashflow = stock.cashflow

        if financials.empty:
            return None

        years = financials.columns[:5] if len(financials.columns) >= 5 else financials.columns

        growth_data = {
            'Ticker': ticker,
            'Years': [year.year for year in years],
            'Revenue': [],
            'Operating_Income': [],
            'Net_Income': [],
            'Free_Cash_Flow': []
        }

        if 'Total Revenue' in financials.index:
            growth_data['Revenue'] = financials.loc['Total Revenue', years].tolist()

        if 'Operating Income' in financials.index:
            growth_data['Operating_Income'] = financials.loc['Operating Income', years].tolist()

        if 'Net Income' in financials.index:
            growth_data['Net_Income'] = financials.loc['Net Income', years].tolist()

        if 'Free Cash Flow' in cashflow.index:
            growth_data['Free_Cash_Flow'] = cashflow.loc['Free Cash Flow', years].tolist()

        return growth_data

    except Exception as e:
        return None

def calculate_cagr(start_value, end_value, years):
    """CAGR 계산"""
    if start_value <= 0 or end_value <= 0 or years <= 0:
        return 0
    try:
        cagr = (pow(end_value / start_value, 1 / years) - 1) * 100
        return round(cagr, 2)
    except:
        return 0

print("\n5개년 재무 데이터 수집 중...\n")

all_growth_data = []

for ticker in mag7_tickers:
    print(f"수집 중: {ticker}...", end=" ")
    growth_data = get_5year_growth_metrics(ticker)

    if growth_data:
        all_growth_data.append(growth_data)
        print(f"✓ ({len(growth_data['Years'])}개년)")
    else:
        print("✗")

if all_growth_data:
    print(f"\n✓ 총 {len(all_growth_data)}개 종목 데이터 수집 완료\n")

    # 매출 성장 차트
    fig_revenue = go.Figure()

    for data in all_growth_data:
        if data['Revenue']:
            revenue_billions = [val / 1e9 for val in reversed(data['Revenue'])]
            years_reversed = list(reversed(data['Years']))

            fig_revenue.add_trace(go.Scatter(
                x=years_reversed,
                y=revenue_billions,
                mode='lines+markers',
                name=data['Ticker'],
                line=dict(width=3),
                marker=dict(size=10)
            ))

    fig_revenue.update_layout(
        title='📊 MAG 7 - 5개년 매출 성장 추이',
        xaxis_title='연도',
        yaxis_title='매출 (Billion USD)',
        hovermode='x unified',
        height=600,
        template='plotly_white'
    )

    fig_revenue.show()

    # 순이익 성장 차트
    fig_income = go.Figure()

    for data in all_growth_data:
        if data['Net_Income']:
            income_billions = [val / 1e9 for val in reversed(data['Net_Income'])]
            years_reversed = list(reversed(data['Years']))

            fig_income.add_trace(go.Scatter(
                x=years_reversed,
                y=income_billions,
                mode='lines+markers',
                name=data['Ticker'],
                line=dict(width=3),
                marker=dict(size=10)
            ))

    fig_income.update_layout(
        title='💰 MAG 7 - 5개년 순이익 성장 추이',
        xaxis_title='연도',
        yaxis_title='순이익 (Billion USD)',
        hovermode='x unified',
        height=600,
        template='plotly_white'
    )

    fig_income.show()

    # CAGR 계산
    print("\n" + "=" * 80)
    print("📊 MAG 7 - 5개년 연평균 성장률 (CAGR)")
    print("=" * 80)

    cagr_summary = []

    for data in all_growth_data:
        ticker = data['Ticker']
        years_count = len(data['Years']) - 1

        if years_count <= 0:
            continue

        cagr_data = {'Ticker': ticker}

        if data['Revenue'] and len(data['Revenue']) >= 2:
            cagr_data['매출_CAGR_%'] = calculate_cagr(
                data['Revenue'][-1], data['Revenue'][0], years_count)
        else:
            cagr_data['매출_CAGR_%'] = 'N/A'

        if data['Net_Income'] and len(data['Net_Income']) >= 2:
            start_ni = data['Net_Income'][-1]
            end_ni = data['Net_Income'][0]
            if start_ni > 0 and end_ni > 0:
                cagr_data['순이익_CAGR_%'] = calculate_cagr(start_ni, end_ni, years_count)
            else:
                cagr_data['순이익_CAGR_%'] = 'N/A'
        else:
            cagr_data['순이익_CAGR_%'] = 'N/A'

        cagr_data['분석기간'] = f"{data['Years'][-1]}-{data['Years'][0]}"
        cagr_summary.append(cagr_data)

    df_cagr = pd.DataFrame(cagr_summary)
    print("\n")
    print(df_cagr.to_string(index=False))

    # CAGR 비교 차트
    fig_cagr = go.Figure()

    revenue_cagr = []
    income_cagr = []
    tickers_cagr = []

    for _, row in df_cagr.iterrows():
        if row['매출_CAGR_%'] != 'N/A':
            revenue_cagr.append(row['매출_CAGR_%'])
            tickers_cagr.append(row['Ticker'])

    fig_cagr.add_trace(go.Bar(
        name='매출 CAGR',
        x=tickers_cagr,
        y=revenue_cagr,
        text=[f"{v:.1f}%" for v in revenue_cagr],
        textposition='auto',
        marker_color='#2196F3'
    ))

    income_cagr = []
    for _, row in df_cagr.iterrows():
        if row['순이익_CAGR_%'] != 'N/A':
            income_cagr.append(row['순이익_CAGR_%'])

    if len(income_cagr) == len(tickers_cagr):
        fig_cagr.add_trace(go.Bar(
            name='순이익 CAGR',
            x=tickers_cagr,
            y=income_cagr,
            text=[f"{v:.1f}%" for v in income_cagr],
            textposition='auto',
            marker_color='#FF9800'
        ))

    fig_cagr.update_layout(
        title='📊 MAG 7 - 5개년 연평균 성장률 (CAGR) 비교',
        xaxis_title='종목',
        yaxis_title='CAGR (%)',
        barmode='group',
        height=600,
        template='plotly_white'
    )

    fig_cagr.show()

# 최종 요약
print("\n" + "=" * 80)
print("📈 MAG 7 투자 전략 최종 요약")
print("=" * 80)

print(f"\n📍 분석 기준: Anchored VWAP ({quarter_start.year} Q{quarter_num})")
print(f"📅 Anchor Point: {quarter_start.strftime('%Y-%m-%d')}")
print(f"🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)")

above_vwap = df_results[df_results['Is_Above_VWAP'] == True]
print(f"✅ VWAP 위 종목: {len(above_vwap)}개")

strong_buy = df_results[df_results['Buy_Signal_Score'] >= 80]
print(f"💚 강력 매수 (80점↑): {len(strong_buy)}개")
if not strong_buy.empty:
    print(f"   → {', '.join(strong_buy['Ticker'].tolist())}")

good_buy = df_results[(df_results['Buy_Signal_Score'] >= 60) &
                      (df_results['Buy_Signal_Score'] < 80)]
print(f"💛 눌림목 대기 (60-80점): {len(good_buy)}개")
if not good_buy.empty:
    print(f"   → {', '.join(good_buy['Ticker'].tolist())}")

watch = df_results[df_results['Buy_Signal_Score'] < 60]
print(f"💙 관찰 필요 (60점 미만): {len(watch)}개")
if not watch.empty:
    print(f"   → {', '.join(watch['Ticker'].tolist())}")

print("\n🏆 TOP 3 추천:")
for idx in range(min(3, len(df_results))):
    row = df_results.iloc[idx]
    print(f"  {idx+1}. {row['Ticker']} - {row['Company']}")
    print(f"     💰 시총: ${row['Market_Cap_Trillion']:.2f}T | " +
          f"점수: {row['Buy_Signal_Score']}/100 | " +
          f"분기수익: {row['Quarter_Return_%']:+.2f}%")

print("\n💡 투자 가이드:")
print("""
1. 💚 강력 매수 (80점 이상):
   - VWAP 위에서 안정적, 즉시 매수 검토
   - VWAP +5% 이상이면 눌림목 대기 권장

2. 💛 눌림목 대기 (60-80점):
   - VWAP 근처 조정 시 매수 기회
   - 손절라인: VWAP -2%

3. 💙 관찰 필요 (60점 미만):
   - VWAP 돌파 확인 후 재검토
   - 펀더멘털과 병행 분석 필수
""")

print("=" * 80)
print("✨ MAGNIFICENT SEVEN 분석 완료!")
print("=" * 80)

🌟 MAGNIFICENT SEVEN (MAG 7) 실시간 종합 분석

빅테크 7대 기업 리스트:
  • AAPL   - Apple Inc.                     | 아이폰, 생태계, 온디바이스 AI
  • MSFT   - Microsoft Corporation          | 클라우드(Azure), 생성형 AI (OpenAI 대주주)
  • GOOGL  - Alphabet Inc.                  | 구글 검색, 유튜브, AI (Gemini)
  • AMZN   - Amazon.com Inc.                | 전자상거래, 클라우드(AWS) 1위
  • NVDA   - NVIDIA Corporation             | AI 반도체(GPU) 독점적 지배자
  • META   - Meta Platforms Inc.            | 페이스북, 인스타그램, AI(Llama)
  • TSLA   - Tesla Inc.                     | 전기차, 자율주행, 로봇

🎯 Anchored VWAP 분석: 2025 Q4
📍 Anchor Point: 2025-10-01
📅 현재까지 거래일: 54일

🔍 MAG 7 종목 분석 중...

분석 중: AAPL (Apple Inc.)... ✓
분석 중: MSFT (Microsoft Corporation)... ✓
분석 중: GOOGL (Alphabet Inc.)... ✓
분석 중: AMZN (Amazon.com Inc.)... ✓
분석 중: NVDA (NVIDIA Corporation)... ✓
분석 중: META (Meta Platforms Inc.)... ✓
분석 중: TSLA (Tesla Inc.)... ✓

📊 MAGNIFICENT SEVEN - 실시간 분석 결과
Ticker               Company                      Description  Current_Price  Anchored_VWAP  Price_vs_VWA


📈 MAG 7 종목별 1년 차트 생성 중...

[1/7] 차트 생성: GOOGL... 

✓

[2/7] 차트 생성: AAPL... 

✓

[3/7] 차트 생성: AMZN... 

✓

[4/7] 차트 생성: NVDA... 

...

✓

[7/7] 차트 생성: META... 

✓

📊 MAG 7 수익률 비교 차트 생성 중...


✓ 비교 차트 완성

📈 MAG 7 - 최근 5개년 성장률 분석

5개년 재무 데이터 수집 중...

수집 중: AAPL... ✓ (5개년)
수집 중: MSFT... ✓ (4개년)
수집 중: GOOGL... ✓ (4개년)
수집 중: AMZN... ✗
수집 중: NVDA... ✓ (4개년)
수집 중: META... ✓ (5개년)
수집 중: TSLA... ✓ (5개년)

✓ 총 6개 종목 데이터 수집 완료




📊 MAG 7 - 5개년 연평균 성장률 (CAGR)


Ticker  매출_CAGR_% 순이익_CAGR_%      분석기간
  AAPL        NaN        N/A 2021-2025
  MSFT      12.42      11.87 2022-2025
 GOOGL      10.75       9.61 2021-2024
  NVDA      69.25      95.51 2022-2025
  META        NaN        N/A 2020-2024
  TSLA        NaN        N/A 2020-2024



📈 MAG 7 투자 전략 최종 요약

📍 분석 기준: Anchored VWAP (2025 Q4)
📅 Anchor Point: 2025-10-01
🌟 분석 대상: MAGNIFICENT SEVEN (7개 종목)
✅ VWAP 위 종목: 2개
💚 강력 매수 (80점↑): 1개
   → GOOGL
💛 눌림목 대기 (60-80점): 1개
   → AAPL
💙 관찰 필요 (60점 미만): 5개
   → AMZN, NVDA, TSLA, MSFT, META

🏆 TOP 3 추천:
  1. GOOGL - Alphabet Inc.
     💰 시총: $3.63T | 점수: 80/100 | 분기수익: +22.36%
  2. AAPL - Apple Inc.
     💰 시총: $4.03T | 점수: 65/100 | 분기수익: +6.38%
  3. AMZN - Amazon.com Inc.
     💰 시총: $2.36T | 점수: 35/100 | 분기수익: +0.03%

💡 투자 가이드:

1. 💚 강력 매수 (80점 이상):
   - VWAP 위에서 안정적, 즉시 매수 검토
   - VWAP +5% 이상이면 눌림목 대기 권장

2. 💛 눌림목 대기 (60-80점):
   - VWAP 근처 조정 시 매수 기회
   - 손절라인: VWAP -2%

3. 💙 관찰 필요 (60점 미만):
   - VWAP 돌파 확인 후 재검토
   - 펀더멘털과 병행 분석 필수

✨ MAGNIFICENT SEVEN 분석 완료!
